In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asf_search as asf
import logging

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
# results = asf.product_search('S1_181296_IW1_20220219T125501_VV_10AF-BURST')
reference = results[0]

args = asf.ASFSearchOptions(
    **{"start": '2022-02-10', "end": '2022-07-01'}
)
s = reference.stack(args)

### Create a Pair object from 2 products

In [3]:
pair = asf.Pair(reference, s[2])

print(f"pair.ref.properties['sceneName']: {pair.ref.properties['sceneName']}")
print(f"pair.ref_time: {pair.ref_time}\n")

print(f"pair.sec.properties['sceneName']: {pair.sec.properties['sceneName']}")
print(f"pair.sec_time: {pair.sec_time}\n")

print(f"pair.temporal_baseline: {pair.temporal_baseline}")
print(f"pair.perpendicular_baseline: {pair.perpendicular_baseline}")

pair.ref.properties['sceneName']: S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252
pair.ref_time: 2022-02-15 22:51:19+00:00

pair.sec.properties['sceneName']: S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E
pair.sec_time: 2022-03-11 22:51:19+00:00

pair.temporal_baseline: 24 days, 0:00:00
pair.perpendicular_baseline: -73


### Check the estimated coherence of the pair

In [4]:
pair.estimate_s1_mean_coherence()

16.952704858593183

### Create a new pair with a long temporal baseline

In [5]:
long_pair = pair = asf.Pair(reference, s[10])
print(f"long_pair.temporal: {long_pair.temporal_baseline}")

long_pair.temporal: 120 days, 0:00:00


### Since the temporal baseline is greater than 48, an exception is raised when checking coherence

In [6]:
long_pair.estimate_s1_mean_coherence()

CoherenceEstimationError: Coherence dataset includes temporal baselines up to 48 days.
                   Temporal baseline: 120 days

### Demonstrate pair equivalence

Two pairs are equal if they both share the same reference and secondary scene IDs.

Using scene IDs for equivalence guarantees that not only are the dates identical, but that the data are too. Data products are sometimes superceded, and until the old data are removed, there may be multiple versions of a product in the archive. 

In [7]:
from copy import deepcopy

a = deepcopy(pair)
b = deepcopy(pair)
c = asf.Pair(reference, s[1])
print(f"a.ref_time: {a.ref_time}, a.sec_time: {a.sec_time}")
print(f"b.ref_time: {b.ref_time}, b.sec_time: {b.sec_time}")
print(f"c.ref_time: {c.ref_time}, c.sec_time: {c.sec_time}")
print(f"a == b: {a == b}")
print(f"a == c: {a == c}")

a.ref_time: 2022-02-15 22:51:19+00:00, a.sec_time: 2022-06-15 22:51:24+00:00
b.ref_time: 2022-02-15 22:51:19+00:00, b.sec_time: 2022-06-15 22:51:24+00:00
c.ref_time: 2022-02-15 22:51:19+00:00, c.sec_time: 2022-02-27 22:51:19+00:00
a == b: True
a == c: False


### Show \_\_repr\_\_

In [8]:
print(f"ref: {a.ref.properties['sceneName']}")
print(f"sec: {c.ref.properties['sceneName']}")
print(f"Pair: {c}")

ref: S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252
sec: S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252
Pair: Pair(S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E)


### \_\_hash\_\_ method

In [9]:
print(hash(a) == hash(b))
print(hash(a) == hash(c))

True
False
